# B929164 Machine Learning CW1

This project aims to create a machine learning model that could determine if a customer would be a good fit to receive a line of credit or not based on a number of relevant financial features.

The data set being used can be found [here](https://www.kaggle.com/c/home-credit-default-risk/data). First published by Hans Hofmann, it contains financial information about german customers from 1994.

A machine learning model that can predict whether a customer should be given credit can be useful for several reasons, lending institutions, such as banks or credit card companies, can use this model to assess the risk associated with providing credit to a particular customer by analysing various factors and patterns in the customer's data. This information helps lenders make informed decisions about granting credit and setting appropriate terms and interest rates. Additionally it can also improve efficiency by automating the process of assessing credit risk.

Furthermore, it can also benefit the customer as customers can use this model to assess their own credit risk and make informed decisions about applying for credit without having to go through the process of applying for credit and being rejected which can have a negative impact on their credit score. It can also eliminate bias that may be present in human decision making.

# Importing Libraries

In [ ]:
# [Imports]

# General purpose data manipulation libraries
import pandas as pd
import numpy as np

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# File system management
import os
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

